In [3]:
import numpy as np
import  pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

/home/mickael/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


#  Load Data
First, let's load thetwo bigram file and createa dictionary and reverse dictionary to translate position and characters 

Bi Gram file are composed of 28 row and columns. 
- Row 1: Initial state
- Row2-27: Alphabet characters (a to z)
- Row 28 : final state, end of the word

In [1]:
file_eng = 'bigramenglish.txt'
file_fr = 'bigramfrancais.txt'

In [4]:
eng = np.loadtxt(file_eng)
fr = np.loadtxt(file_fr)

In [8]:
print("Shape of english bigram: {}".format(eng.shape))

Shape of english bigram: (28, 28)


In [5]:
dic = {'1' : ' ', '2' : 'a', '3' : 'b', '4': 'c', '5' : 'd', '6' : 'e', '7': 'f',  '8' : 'g', '9' : 'h', '10': 'i', '11': 'j', '12' : 'k', '13' : 'l', '14': 'm', '15' : 'n', '16' : 'o', '17': 'p', '18' : 'q', '19' : 'r' , '20': 's',  '21' : 't', '22' : 'u', '23': 'v',  '24' : 'w', '25' : 'x' , '26': 'y',  '27' : 'z', '28' : ' ' }

In [6]:
dic_inv = {v: k for k, v in dic.items()}

# Part 1 - Word Generation

In [9]:
# Function which given a vec of the bi-gram will ramdomly choose the next element
def next_el(vec):
    t = np.random.random()
    v = np.cumsum(vec)
    idx = np.sum(v < t)
    return idx

In [12]:
# Generate a word based on the be gram. Can be put a initial word sence 
def generate_el(proba, init = ' '):
    el = []
    if init != ' ':
        el += init[:]
        init = init[-1]
        idx = int(dic_inv[init])-1
    else : 
        idx = 0
    while idx != 27 :
        idx = next_el(proba[idx])
        el += [dic[str(idx+1)]] 
    return "".join(el)

In [15]:
for i in np.arange(20):
    a = generate_el(eng, "th")
    print(a)

there 
then 
th 
the 
thraprererse 
the 
the 
thof 
thig 
the 
thiepledane 
thexucourshts 
theeds 
thedmathers 
th 
ther 
the 
thicorain 
therobm 
thed 


We noticed that some of the word does not mean anything. It is normal, the function based itself on the probability of the sentence of characters but it does ensure that the  word is correct. 

# Part 2 - Sentence Generation

To update our bigram for Sentence Generation, we will do two things: 
- Add a row: The new row (row 29) represent the end of the sentence.
- Update row 28 (end of word) : The row corresponding to the end of sentence to give two new possibilites, the possibility to go back to state 1 (beginning of new word) or go to state 29 (end of sentence).

In [16]:
#update dictionnary to have caracter 29 . 
d = {'29' : '.'}
dic.update(d)

In [18]:
# Create a function that will change the bigram matrix and update the matrix as described above
# In this case, we choose 0.9 as the probability to start a new word at the end of a word
# We chose as 0.1 the probability to stop a sentence at the end of a word
def modify_mat_dic(mat): 
    A = mat.copy()
    n = A.shape[0]
    new_mat = np.zeros((n+1,n+1))
    new_mat[:-1,:-1] = A
    new_mat[-1,-1] = 1 
    new_mat[-2,0] = 0.9
    new_mat[-2,-2] = 0
    new_mat[-2,-1] = 0.1
    return new_mat

In [19]:
def generate_seq_el(proba, init = ' '):
    el = []
    if init != ' ':
        el += init[:]
        init = init[-1]
        idx = int(dic_inv[init])-1
    else : 
        idx = 0
    while idx != 28 :
        idx = next_el(proba[idx])
        el += [dic[str(idx+1)]] 
    return "".join(el)

In [20]:
eng_modify = modify_mat_dic(eng) 
fr_modify = modify_mat_dic(fr)

In [21]:
for i in np.arange(5):
    a = generate_seq_el(eng_modify, "th")
    print(a)

the  or .
thritens  tharion  ss  anofen  ofeprhe  ithe  todinstigeduibar  t  ibjol  t  cedut  s  woly  thilyeaigebo  n  nathe  o  oon  sthand  duesco  by  thee  mesll  anaprnct  ororgly  a  o  ous  moronthac  bint  win  ispang  d  feedesacin  atrdeis  os  an  asen .
they  tirs  st  sfuthee  agln  inis  baprs  tuxic  fur  a .
tho  ioverey  d  be .
thellipe  tick  athey  emay  hed  aly .


As we saw in word generation, the sentence does not mean anything as well, because they are based on probabilities only. 
At this point, we might question the  usefulness of such thing. The previous two part where just a way to manipulate and understand how a bigram  works and markov chain. The part 3 will give a practical example of how to use those bigrams

# Part 3 - Language recognition

In this part, we use the bigram to test the likelyhood of a given sentence for a given language. The strongest likelyhood will be our prediction language. 
For example, we will check the likelyhood of the sentence "be or not to be." in French and English. We will see if the algorithm recognize the language. 

First, in order to better test more easily a sentence, a small transformation need to be made. Let's add the character '-' before each word and + at the end of each word : 
- '-' : Correspond to the first state character 
- '+' : Correspond to the end of word state

Therefore a sentence like "be or not to be." becomes "-be+-or+-not+-to+-be+."
We suppose that every sentence will finish with the character "."

In [29]:
def  transform_sentence(sentence): 
    s = ""
    l = sentence.split(" ")
    for el in l : 
        s += "+"+el+"-"
    return s

In [30]:
test ="to be or not"
transform_sentence(test)

'+to-+be-+or-+not-'

In [31]:
# let's  create a new reverse dictionary with those updated  characters. 
# This reverse dictionary will make the translation between the sentence and the bigram matrix.
dic_vrai = dic_inv
d1 = {'+':'1', '-':'28'}
dic_vrai.update(d1)
dic_vrai.pop(" ")

KeyError: ' '

In [32]:
dic_vrai

{'a': '2',
 'b': '3',
 'c': '4',
 'd': '5',
 'e': '6',
 'f': '7',
 'g': '8',
 'h': '9',
 'i': '10',
 'j': '11',
 'k': '12',
 'l': '13',
 'm': '14',
 'n': '15',
 'o': '16',
 'p': '17',
 'q': '18',
 'r': '19',
 's': '20',
 't': '21',
 'u': '22',
 'v': '23',
 'w': '24',
 'x': '25',
 'y': '26',
 'z': '27',
 '+': '1',
 '-': '28'}

Now, let's createa function that will read the given string and calculate its likelyhood in French and English. 
To understand this calculation, we have to consider the sentence as a markov chain. 
Therefore given a sentence S, we assume S = $(q_1, q_2, ... q_n)$ where each $q_i \in ["+","-",".","a","b",..."z"]$

The likelyhood will be then calculate by $L(S) = \Pi_{i=1}^n P(X_{i+1} = q_{i+1} | X_i = q_i)$

In [25]:
# unit functions that will give you the probability P(X_i+i | X_i)
def eng_vrai_next_el(init_el, next_el):
    idx_init = int(dic_vrai[init_el]) - 1
    idx_next = int(dic_vrai[next_el]) - 1 
    return eng_modify[idx_init, idx_next]

def fr_vrai_next_el(init_el, next_el):
    idx_init = int(dic_vrai[init_el]) - 1
    idx_next = int(dic_vrai[next_el]) - 1 
    return fr_modify[idx_init, idx_next]

In [26]:
# functions that calculate the likelyhood for french and english 
def eng_vraisemblance(sentence): 
    vr = 1 
    sentence_tr = transform_sentence(sentence)
    for i in range(len(sentence_tr)-1):
        vr = vr * eng_vrai_next_el(sentence_tr[i], sentence_tr[i+1])
    return vr

def fr_vraisemblance(sentence): 
    vr = 1 
    sentence_tr = transform_sentence(sentence)
    for i in range(len(sentence_tr)-1):
        vr = vr * fr_vrai_next_el(sentence_tr[i], sentence_tr[i+1])
    return vr

In [27]:
# function that tells predict if a sentence is french or english
def language_recognition(sentence): 
    if eng_vraisemblance(sentence) > fr_vraisemblance(sentence):
        print("It is a English sentence")
    else : 
        print("It is a French sentence")

In [33]:
language_recognition("to be or not to be")

It is a English sentence


In [34]:
language_recognition("etre ou ne pas etre")

It is a French sentence
